# Introduction to Synthetic Aperture Radar Using Python and MATLAB

## by Andy Harrison - &copy; Artech House 2022

---

## Example 6.8.4 ORB
---

**Import modules**

In [1]:
import cv2

import numpy as np

**Helper function for displaying images**

In [2]:
def imshow(image, windowName, nx, ny):
    cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)    # Create a new named window
    cv2.moveWindow(windowName, 0, 0)                  # Put window @ (0, 0)
    cv2.imshow(windowName, image)                     # Display the image
    cv2.resizeWindow(windowName, nx, ny)              # Resize the window

**Read the original image**

In [3]:
image1 = cv2.imread('ICEYE_ORB.jpg')

**Define the source and destination triangles**

In [4]:
srcTri = np.array( [[0, 0], [image1.shape[1] - 1, 0], [0, image1.shape[0] - 1]] ).astype(np.float32)

dstTri = np.array( [[0, image1.shape[1]*0.1], [image1.shape[1]*0.78, image1.shape[0]*0.1], [image1.shape[1]*0.2, image1.shape[0]*0.85]] ).astype(np.float32)

**Get the affine transform**

In [5]:
warp_mat = cv2.getAffineTransform(srcTri, dstTri)

warp_dst = cv2.warpAffine(image1, warp_mat, (image1.shape[1], image1.shape[0]))

**Rotating the image after Warp**

In [6]:
center = (warp_dst.shape[1] // 2, warp_dst.shape[0] // 2)

angle = 17

scale = 0.86

rot_mat = cv2.getRotationMatrix2D(center, angle, scale)

warp_rotate_dst = cv2.warpAffine(warp_dst, rot_mat, (warp_dst.shape[1], warp_dst.shape[0]))

**Display the original and warped images**

In [7]:
imshow(image1, 'Source Image', 600, 600)

imshow(warp_rotate_dst, 'Warped Image', 600, 600)

image1s = image1

**Convert images to grayscale**

In [8]:
image2 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

image1 = cv2.cvtColor(warp_rotate_dst, cv2.COLOR_BGR2GRAY)


# ORB

orb = cv2.ORB_create()

**Find the keypoints with ORB**

In [9]:
kp1 = orb.detect(image1, None)

kp2 = orb.detect(image2, None)

**compute the descriptors with ORB**

In [10]:
keypoints_1, descriptors_1 = orb.compute(image1, kp1)

keypoints_2, descriptors_2 = orb.compute(image2, kp2)

**Find feature matches**

In [11]:
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1, descriptors_2)

matches = sorted(matches, key=lambda x: x.distance)

**Display the matches**

In [12]:
image3 = cv2.drawMatches(warp_rotate_dst, keypoints_1, image1s, keypoints_2, matches[:20], image2, flags=2)

imshow(image3, 'Matches', 600, 600)

**Get the coordinates of the keypoints**

In [13]:
list_kp1 = np.float32([keypoints_1[mat.queryIdx].pt for mat in matches]).reshape(-1, 1, 2)

list_kp2 = np.float32([keypoints_2[mat.trainIdx].pt for mat in matches]).reshape(-1, 1, 2)

**With coordinates of keypoints, find homography**

In [14]:
h, status = cv2.findHomography(list_kp1[0:5], list_kp2[0:5], cv2.RANSAC, 5.0)

**Warp source image to destination based on homography**

In [ ]:
image3 = cv2.warpPerspective(warp_rotate_dst, h, (image2.shape[1], image2.shape[0]))

imshow(image3, 'Final Image', 600, 600)

cv2.waitKey()